<div class="alert alert-block alert-danger">

# FIT5196 Task 2 in Assessment 1
    
#### Student Name: Namzus Sakib
#### Student ID: 33361991

Date: 27th August 2023

Environment: Python 3.9

Libraries used:
* os (for interacting with the operating system, included in Python xxxx)
* pandas 1.1.0 (for dataframe, installed and imported)
* multiprocessing (for performing processes on multi cores, included in Python 3.6.9 package)
* itertools (for performing operations on iterables)
* nltk 3.5 (Natural Language Toolkit, installed and imported)
* nltk.collocations (for finding bigrams, installed and imported)
* nltk.tokenize (for tokenization, installed and imported)
* nltk.stem (for stemming the tokens, installed and imported)

    </div>

<div class="alert alert-block alert-info">
    
## Table of Contents

</div>

[1. Introduction](#Intro) <br>
[2. Importing Libraries](#libs) <br>
[3. Examining Input File](#examine) <br>
[4. Loading and Parsing Files](#load) <br>
$\;\;\;\;$[4.1. Tokenization](#tokenize) <br>
$\;\;\;\;$[4.2. Whatever else](#whetev) <br>
$\;\;\;\;$[4.3. Finding First 200 Bigrams](#bigrams) <br>
$\;\;\;\;$[4.4. Whatever else](#whetev1) <br>
[5. Writing Output Files](#write) <br>
$\;\;\;\;$[5.1. Vocabulary List](#write-vocab) <br>
$\;\;\;\;$[5.2. Sparse Matrix](#write-sparseMat) <br>
[6. Summary](#summary) <br>
[7. References](#Ref) <br>

<div class="alert alert-block alert-success">
    
## 1.  Introduction  <a class="anchor" name="Intro"></a>

This assessment concerns textual data and the aim is to extract data, process them, and transform them into a proper format. The dataset provided is in the format of a PDF file containing ....

<div class="alert alert-block alert-success">
    
## 2.  Importing Libraries  <a class="anchor" name="libs"></a>

In this assessment, any python packages is permitted to be used. The following packages were used to accomplish the related tasks:

* **os:** to interact with the operating system, e.g. navigate through folders to read files
* **re:** to define and use regular expressions
* **pandas:** to work with dataframes
* **multiprocessing:** to perform processes on multi cores for fast performance
* ...

In [56]:
!pip install langid
!pip install pandas PyPDF2
!pip install PyMuPDF
!pip install tabula-py
!pip install requests beautifulsoup4
!pip install pdfplumber

from sklearn.feature_extraction.text import CountVectorizer
from collections import Counter
import pdfplumber
import requests
from bs4 import BeautifulSoup
from tabula import read_pdf
import fitz
import PyPDF2
import os
import re
import langid
import pandas as pd
import multiprocessing
from itertools import chain
import nltk
from nltk.corpus import stopwords
from nltk.probability import *
from nltk.collocations import *
from nltk.tokenize import RegexpTokenizer
from nltk.tokenize import MWETokenizer
from nltk.stem import PorterStemmer
from nltk.util import ngrams

-------------------------------------

<div class="alert alert-block alert-success">
    
## 3.  Examining Input File <a class="anchor" name="examine"></a>

In [57]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


Let's examine what is the content of the file. For this purpose, the content will be read through file path and I have checked it before that it will contain 2 columns which are called id and download link respectively. After the file is read and stored in data_df dataframe, I will write in to csv file with the name 33361991_paper_list.csv

In [58]:
# Path to the PDF file
file_path = '/content/drive/Shareddrives/FIT5196_S2_2023/Assessment 1/student_data/task2/33361991_task2_input.pdf'

# Extract tabular data from the PDF using tabula-py
df_list = read_pdf(file_path, pages='all', multiple_tables=True)

# Assuming the relevant table is the first one
# If needed, you can iterate through df_list to identify the correct table
data_df = df_list[0]

# Rename columns to match your expected columns
data_df.columns = ['id', 'download_link']

# Display the DataFrame
print(data_df)

# Save the DataFrame to a CSV file named '33361991_paper_list.csv'
data_df.to_csv('33361991_paper_list.csv', index=False)

       id                                      download_link
0    5483  https://papers.nips.cc/paper_files/paper/2022/...
1   10026  https://cdn.aaai.org/ojs/17611/17611-13-21105-...
2    1969  https://papers.nips.cc/paper_files/paper/2021/...
3    4055  https://papers.nips.cc/paper_files/paper/2021/...
4    5086  https://papers.nips.cc/paper_files/paper/2022/...
..    ...                                                ...
75   9501  https://cdn.aaai.org/ojs/17098/17098-13-20592-...
76    448  https://papers.nips.cc/paper_files/paper/2020/...
77   7757  https://cdn.aaai.org/ojs/5722/5722-13-8947-1-1...
78   9374  https://cdn.aaai.org/ojs/17021/17021-13-20515-...
79   1098  https://papers.nips.cc/paper_files/paper/2020/...

[80 rows x 2 columns]


It is noteiced that file contains id and links to the papers.

Having parsed the pdf file, the following observations can be made:
1. The pdf has 80 rows and 2 columns which contains the IDs and the link to the research paper


<div class="alert alert-block alert-success">
    
## 4.  Loading and Parsing File <a class="anchor" name="load"></a>

In this section, I have created a folder to store the downloaded research papers. And iterate through each of the download links to downlaod the pdf in the research_papers folder with the name as its id.

In [59]:
# Create a folder to save downloaded research papers
output_folder = 'research_papers'
if not os.path.exists(output_folder):
    os.makedirs(output_folder)

In [60]:
# Iterate through the DataFrame and download research papers
for index, row in data_df.iterrows():
    download_url = row['download_link']
    paper_id = row['id']

    response = requests.get(download_url)
    paper_path = os.path.join(output_folder, f'{paper_id}_paper.pdf')

    with open(paper_path, 'wb') as paper_file:
        paper_file.write(response.content)

    print(f"Downloaded {paper_id}_paper.pdf")

print("Download completed.")

Downloaded 5483_paper.pdf
Downloaded 10026_paper.pdf
Downloaded 1969_paper.pdf
Downloaded 4055_paper.pdf
Downloaded 5086_paper.pdf
Downloaded 956_paper.pdf
Downloaded 8886_paper.pdf
Downloaded 10675_paper.pdf
Downloaded 10522_paper.pdf
Downloaded 1672_paper.pdf
Downloaded 7127_paper.pdf
Downloaded 129_paper.pdf
Downloaded 5445_paper.pdf
Downloaded 477_paper.pdf
Downloaded 2058_paper.pdf
Downloaded 2409_paper.pdf
Downloaded 4392_paper.pdf
Downloaded 6916_paper.pdf
Downloaded 4531_paper.pdf
Downloaded 540_paper.pdf
Downloaded 5922_paper.pdf
Downloaded 8443_paper.pdf
Downloaded 2278_paper.pdf
Downloaded 679_paper.pdf
Downloaded 5639_paper.pdf
Downloaded 1808_paper.pdf
Downloaded 9778_paper.pdf
Downloaded 2613_paper.pdf
Downloaded 1190_paper.pdf
Downloaded 8817_paper.pdf
Downloaded 1638_paper.pdf
Downloaded 2715_paper.pdf
Downloaded 5953_paper.pdf
Downloaded 3177_paper.pdf
Downloaded 8492_paper.pdf
Downloaded 6048_paper.pdf
Downloaded 4654_paper.pdf
Downloaded 1006_paper.pdf
Downloaded 102

How we can see that all 80 of the research papers are downloaded and saved in the research_papers folder.

Now I will create an empty list abstract then iterate through all the papers to find the abstract part and store it in the abtract list. The abstract should be stored according to the id of the paper. The abstract will be extracted through the proper regex


In [61]:
# Iterate through the downloaded papers and extract the 'Abstract'
abstracts = []

for index, row in data_df.iterrows():
    paper_id = row['id']
    paper_path = os.path.join(output_folder, f'{paper_id}_paper.pdf')

    with pdfplumber.open(paper_path) as pdf:
        # Extract text from the first page (you might need to adjust this)
        first_page = pdf.pages[0]
        text = first_page.extract_text()

        # Extract the 'Abstract' section (you might need to adjust this)
        abstract_match = re.search(r'\bAbstract\b(.+?)\bIntroduction\b', text, re.DOTALL)
        if abstract_match:
            abstract = abstract_match.group(1).strip()

            # Split text by line breaks and join with a space
            abstract = ' '.join(abstract.splitlines())

            abstracts.append(abstract)
        else:
            abstracts.append("Abstract not found")

# Add the 'Abstract' column to the DataFrame
data_df['abstract'] = abstracts

# Display the DataFrame with 'Abstract' column
print(data_df)

       id                                      download_link  \
0    5483  https://papers.nips.cc/paper_files/paper/2022/...   
1   10026  https://cdn.aaai.org/ojs/17611/17611-13-21105-...   
2    1969  https://papers.nips.cc/paper_files/paper/2021/...   
3    4055  https://papers.nips.cc/paper_files/paper/2021/...   
4    5086  https://papers.nips.cc/paper_files/paper/2022/...   
..    ...                                                ...   
75   9501  https://cdn.aaai.org/ojs/17098/17098-13-20592-...   
76    448  https://papers.nips.cc/paper_files/paper/2020/...   
77   7757  https://cdn.aaai.org/ojs/5722/5722-13-8947-1-1...   
78   9374  https://cdn.aaai.org/ojs/17021/17021-13-20515-...   
79   1098  https://papers.nips.cc/paper_files/paper/2020/...   

                                             abstract  
0   ThekernelMaximumMeanDiscrepancy(MMD)isapopular...  
1   TheprogressinQuery-focusedMulti-DocumentSummar...  
2   Perturb-and-MAPoffersanelegantapproachtoapprox...  
3   Adv

Now we can see that the abstracts are extracted in the abstract column of the dataframe and the abstracts are according to the ids of the paper.

Now I will replace the ligatures, for now I have used the default but kept space for more if needed.

In [62]:
def replace_ligatures(text):
    ligatures = {
        'ﬁ': 'fi', 'ﬂ': 'fl', 'ﬀ': 'ff', 'ﬃ': 'ffi', 'ﬄ': 'ffl',
        # Add more ligatures as needed
    }

    for ligature, replacement in ligatures.items():
        text = text.replace(ligature, replacement)

    return text

Now I will create a function for the replacement of the special characters in the text.

In [63]:
def replace_special_characters(text):
    # Use appropriate replacements for special HTML entities
    special_characters = {
        '&amp;': '&',
        '&lt;': '<',
        '&gt;': '>',
        # Add more special characters as needed
    }

    for entity, replacement in special_characters.items():
        text = text.replace(entity, replacement)

    return text

Now I will create a function for the hyphenated words int he abstract column.

In [64]:
def fix_hyphenated_words(text):
    # Replace hyphen-separated words
    fixed_text = re.sub(r'(?<=\w)-\s*(?=\w)', '', text)
    return fixed_text

Now I will iterate through the abstract column and apply the replace_ligature, replace_special_characters and fix_hyphenated_words for each row and then display the data frame.

In [65]:
# Iterate through the 'Abstract' column and apply replacements
data_df['abstract'] = data_df['abstract'].apply(replace_ligatures)
data_df['abstract'] = data_df['abstract'].apply(replace_special_characters)
data_df['abstract'] = data_df['abstract'].apply(fix_hyphenated_words)

# Display the DataFrame with replaced 'Abstract'
print(data_df)

       id                                      download_link  \
0    5483  https://papers.nips.cc/paper_files/paper/2022/...   
1   10026  https://cdn.aaai.org/ojs/17611/17611-13-21105-...   
2    1969  https://papers.nips.cc/paper_files/paper/2021/...   
3    4055  https://papers.nips.cc/paper_files/paper/2021/...   
4    5086  https://papers.nips.cc/paper_files/paper/2022/...   
..    ...                                                ...   
75   9501  https://cdn.aaai.org/ojs/17098/17098-13-20592-...   
76    448  https://papers.nips.cc/paper_files/paper/2020/...   
77   7757  https://cdn.aaai.org/ojs/5722/5722-13-8947-1-1...   
78   9374  https://cdn.aaai.org/ojs/17021/17021-13-20515-...   
79   1098  https://papers.nips.cc/paper_files/paper/2020/...   

                                             abstract  
0   ThekernelMaximumMeanDiscrepancy(MMD)isapopular...  
1   TheprogressinQueryfocusedMultiDocumentSummariz...  
2   PerturbandMAPoffersanelegantapproachtoapproxim...  
3   Adv

<div class="alert alert-block alert-warning">
    
### 4.1. Tokenization <a class="anchor" name="tokenize"></a>

Tokenization is a principal step in text processing and producing unigrams and bigrams. In this section, I have tokenized the abstract and also read the stopwords_en.txt which was provided to me in moodle.

The tokenization was done using the regex:
[A-Za-z]\w+(?:[-'?]\w+)?

and also PorterStemmer is initialized

Step 1 is the tokenization of the abstrcat column and step 2 is the removal of context-independent stop words

In [66]:
# Load NLTK resources
nltk.download('stopwords')

# Load stopwords_en.txt
with open('stopwords_en.txt', 'r') as stopwords_file:
    context_independent_stopwords = set(stopwords_file.read().splitlines())

# Initialize the Porter stemmer
porter_stemmer = PorterStemmer()

# Tokenization regular expression
token_pattern = r"[A-Za-z]\w+(?:[-'?]\w+)?"

# Step 1: Tokenize using the regular expression
def tokenize(text):
    return re.findall(token_pattern, text.lower())

data_df['tokens'] = data_df['abstract'].apply(tokenize)

# Step 2: Remove context-independent stop words
def remove_stopwords(tokens):
    return [token for token in tokens if token not in context_independent_stopwords]

data_df['tokens'] = data_df['tokens'].apply(remove_stopwords)

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


Now I display the data_df to see how it looks like.

In [67]:
data_df

,id,download_link,abstract,tokens
0,5483,https://papers.nips.cc/paper_files/paper/2022/...,ThekernelMaximumMeanDiscrepancy(MMD)isapopular...,"[thekernelmaximummeandiscrepancy, mmd, isapopu..."
1,10026,https://cdn.aaai.org/ojs/17611/17611-13-21105-...,TheprogressinQueryfocusedMultiDocumentSummariz...,[theprogressinqueryfocusedmultidocumentsummari...
2,1969,https://papers.nips.cc/paper_files/paper/2021/...,PerturbandMAPoffersanelegantapproachtoapproxim...,[perturbandmapoffersanelegantapproachtoapproxi...
3,4055,https://papers.nips.cc/paper_files/paper/2021/...,Adversarial patch attacks that craft the pixel...,"[adversarial, patch, attacks, craft, pixels, c..."
4,5086,https://papers.nips.cc/paper_files/paper/2022/...,Reinforcementlearning(RL)inlonghorizonandspars...,"[reinforcementlearning, rl, inlonghorizonandsp..."
...,...,...,...,...
75,9501,https://cdn.aaai.org/ojs/17098/17098-13-20592-...,"Withtherecentadvancesingraphneuralnetworks,the...","[withtherecentadvancesingraphneuralnetworks, t..."
76,448,https://papers.nips.cc/paper_files/paper/2020/...,Westudyminimaxconvergenceratesofnonparametricd...,[westudyminimaxconvergenceratesofnonparametric...
77,7757,https://cdn.aaai.org/ojs/5722/5722-13-8947-1-1...,significantly advanced the stateoftheart in va...,"[significantly, advanced, stateoftheart, lingu..."
78,9374,https://cdn.aaai.org/ojs/17021/17021-13-20515-...,"query samples (i.e., query set) from the novel...","[query, samples, query, set, set, unseenclasse..."


<div class="alert alert-block alert-warning">
    
### 4.2. Removal of rare tokens <a class="anchor" name="whetev"></a>

Now we remove the rare tokens which apprear in less than 3% of the files

In [68]:
# Remove rare tokens (appearing in less than 3% of the files)
def remove_rare_tokens(tokens, frequency_threshold=0.03):
    total_docs = len(data_df)
    token_counter = Counter(tokens)

    return [token for token in tokens if token_counter[token] / total_docs >= frequency_threshold]

data_df['tokens'] = data_df['tokens'].apply(remove_rare_tokens)

data_df

,id,download_link,abstract,tokens
0,5483,https://papers.nips.cc/paper_files/paper/2022/...,ThekernelMaximumMeanDiscrepancy(MMD)isapopular...,[]
1,10026,https://cdn.aaai.org/ojs/17611/17611-13-21105-...,TheprogressinQueryfocusedMultiDocumentSummariz...,"[dataset, dataset, dataset]"
2,1969,https://papers.nips.cc/paper_files/paper/2021/...,PerturbandMAPoffersanelegantapproachtoapproxim...,[]
3,4055,https://papers.nips.cc/paper_files/paper/2021/...,Adversarial patch attacks that craft the pixel...,[]
4,5086,https://papers.nips.cc/paper_files/paper/2022/...,Reinforcementlearning(RL)inlonghorizonandspars...,[]
...,...,...,...,...
75,9501,https://cdn.aaai.org/ojs/17098/17098-13-20592-...,"Withtherecentadvancesingraphneuralnetworks,the...","[chair, person, chair, person, umbrella, dinin..."
76,448,https://papers.nips.cc/paper_files/paper/2020/...,Westudyminimaxconvergenceratesofnonparametricd...,[]
77,7757,https://cdn.aaai.org/ojs/5722/5722-13-8947-1-1...,significantly advanced the stateoftheart in va...,"[stateoftheart, tasks, al, stateoftheart, mode..."
78,9374,https://cdn.aaai.org/ojs/17021/17021-13-20515-...,"query samples (i.e., query set) from the novel...","[samples, set, set, samples, al, set, al, al, ..."


This section calculates the frequency of tokens and bigrams, and then removes context-dependent stop words (both unigrams and bigrams) based on the appearance frequency threshold of 95% or more using the stemmed tokens.

Also uses token_counter to store the final tokens which are less than 3 characters or symbols.

Final the final_tokens are sorted

In [69]:
vocab_list = []

for tokens in data_df['tokens']:
    stemmed_tokens = [porter_stemmer.stem(token) for token in tokens if len(token) >= 3]
    vocab_list.extend(stemmed_tokens)

# Calculate token and bigram frequencies
token_counter = Counter(vocab_list)
all_bigrams = [bigram for tokens in data_df['tokens'] for bigram in ngrams(tokens, 2)]
bigram_counter = Counter(all_bigrams)

# Calculate the threshold for context-dependent stop words
doc_count_threshold = int(len(data_df) * 0.95)

# Remove context-dependent stop words
final_tokens = []

for token in token_counter:
    if token_counter[token] < doc_count_threshold and len(token) >= 3:
        final_tokens.append(token)

for bigram in bigram_counter:
    if bigram_counter[bigram] < doc_count_threshold:
        # Convert the bigram tuple to a string
        bigram_str = ' '.join(bigram)
        final_tokens.append(bigram_str)

# Sort and remove duplicates from final_tokens
final_tokens = sorted(set(final_tokens))

At this stage, all reviews for each PID are tokenized and are stored as a value in the new dictionary (separetely for each day).

-------------------------------------

<div class="alert alert-block alert-warning">
    
### 4.3. Generate numerical representation<a class="anchor" name="bigrams"></a>

One of the tasks is to generate the numerical representation for all tokens in abstract.

In [70]:
# Create a mapping of token to index
token_to_index = {token: index for index, token in enumerate(final_tokens)}

# Generate the sparse numerical representation using CountVectorizer
vectorizer = CountVectorizer(vocabulary=token_to_index, tokenizer=lambda x: x.split())
sparse_representation = vectorizer.transform(data_df['abstract'])

the CountVectorizer from scikit-learn is used to generate the sparse numerical representation of the token frequencies. The token_to_index mapping ensures that the tokens are mapped to their respective indices, as generated in step 3. The resulting sparse representation is then output to the countvec.txt file as per the specified format.

At this stage, we have a dictionary of tokenized words.

-------------------------------------

<div class="alert alert-block alert-success">
    
## 5. Writing Output Files <a class="anchor" name="write"></a>

files need to be generated:
* Vocabulary list
* Sparse matrix (count_vectors)

This is performed in the following sections.

<div class="alert alert-block alert-warning">
    
### 5.1. Vocabulary List <a class="anchor" name="write-vocab"></a>

List of vocabulary should also be written to a file, sorted alphabetically, with their reference codes in front of them. This file also refers to the sparse matrix in the next file. For this purpose, .....

In [71]:
# Generate and save vocab.txt
with open('33361991_vocab.txt', 'w') as vocab_file:
    for index, token in enumerate(final_tokens):
        vocab_file.write(f"{token}: {index}\n")

<div class="alert alert-block alert-warning">
    
### 5.2. Sparse Matrix <a class="anchor" name="write-sparseMat"></a>

For writing sparse matrix for a paper, we firstly calculate the frequency of words for that paper.
The token frequencies for each paper are calculated and stored in the paper_token_frequencies dictionary. The resulting sparse numerical representation is then output to the 33361991_countvec.txt file as per the specified format.

In [72]:
# Initialize an empty dictionary to store token frequencies per paper
paper_token_frequencies = {}

# Iterate through papers and calculate token frequencies
for index, row in data_df.iterrows():
    paper_tokens = row['tokens']
    token_frequencies = Counter(paper_tokens)
    paper_token_frequencies[row['id']] = token_frequencies

# Write the sparse numerical representation to countvec.txt
with open('33361991_countvec.txt', 'w') as countvec_file:
    for paper_id, token_freqs in paper_token_frequencies.items():
        token_indices_and_frequencies = [
            f"{token_to_index[token]}:{frequency}"
            for token, frequency in token_freqs.items()
            if token in token_to_index
        ]
        countvec_file.write(f"{paper_id}, {' '.join(token_indices_and_frequencies)}\n")

-------------------------------------

<div class="alert alert-block alert-success">
    
## 6. Summary <a class="anchor" name="summary"></a>

Here's a summary of the entire process from top to bottom:

1. **Import Libraries**:

  Import necessary libraries such as os, re, nltk, sklearn, and others for data manipulation, text processing, and generating sparse representations.

2. **Load DataFrame**:

  Load your DataFrame containing 'id' and 'abstract' columns that store paper IDs and abstract texts.

3. **Load NLTK Resource**s:

  Download NLTK resources like stopwords using nltk.download('stopwords').

4. **Load Stopwords**:

  Load a list of context-independent stopwords from a file named 'stopwords_en.txt'.

5. **Initialize Stemmer**:

  Initialize a Porter stemmer for stemming tokens.

6. **Tokenization**:

  Tokenize abstract texts using a regular expression pattern and store the tokens in the DataFrame.

7. **Remove Context-Independent Stopwords**:

  Remove context-independent stopwords from the tokenized abstracts.

8. **Remove Rare Tokens**:

  Remove tokens that appear in less than 3% of the papers.

9. **Preprocessing Steps**:

  Implement and apply additional preprocessing steps, including steps like stemming, removing context-dependent stopwords, removing short tokens, and more.

10. **Generate Vocabulary**:

  Generate a vocabulary list based on the processed tokens, considering various preprocessing steps.

11. **Create Token-to-Index Mapping**:

  Create a mapping of tokens to their corresponding indices in the vocabulary.

12. **Calculate Token Frequencies**:

  Calculate token frequencies for each paper and store them in a dictionary.

13. **Generate Sparse Numerical Representation**:

  Use CountVectorizer from scikit-learn to generate a sparse numerical representation of token frequencies.

14. **Output Sparse Representation**:

  Write the sparse numerical representation to a file named 'countvec.txt', including paper IDs and token frequencies in the specified format.

**Summary**:

The entire process involves loading the data, performing text preprocessing steps, generating sparse representations, and outputting the results to files.

-------------------------------------

<div class="alert alert-block alert-success">
    
## 7. References <a class="anchor" name="Ref"></a>

[1] Pandas dataframe.drop_duplicates(), https://www.geeksforgeeks.org/python-pandas-dataframe-drop_duplicates/, Accessed 13/08/2022.



## --------------------------------------------------------------------------------------------------------------------------